In [1]:
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

print(tf.__version__, tfp.__version__)

2.3.0 0.11.0


In [2]:
# constant term included as column in the matrices

kwargs = {
  'y' : tf.constant([1,2]), # size of T
  'trends' : tf.constant([[1,2],[3,4]]), # size of T * G
  'regressors' : tf.constant([[1,2],[3,4]]), # size of T * J
  'exo_factors' : tf.constant([[1,2],[3,4]]), # size of T * K
  'regularization' : 0.0, # scalar
  'delta' : tf.constant([2,3]) # one dimensional vector
}

In [3]:
def baseline(trends, g):
  return tf.matmul(trends, tf.expand_dims(g, -1))

def short_term(regressors, beta):
  return tf.matmul(regressors, tf.expand_dims(beta, -1))

def long_term(regressors, gamma, delta):
  add_stock = tf.nn.conv2d(
    regressors[tf.newaxis, :, :, tf.newaxis],
    filters=delta[:, tf.newaxis, tf.newaxis, tf.newaxis],
    strides=1,
    padding="VALID"
  )
  
  regressors_accu = tf.math.log(tf.squeeze(add_stock))
  
  return tf.matmul(regressors_accu, tf.expand_dims(gamma, -1))
  
def exo_term(exo_factors, theta):
  return tf.matmul(exo_factors, tf.expand_dims(theta, -1))

In [4]:
@tf.function(experimental_compile=True)
def objective_function(params, y, trends, regressors, exo_factors, regularization, delta):
  # unpack params
  G, J, K, D = trends.shape[1], regressors.shape[1], exo_factors.shape[1], delta.shape[0]
  g = params[:G]
  beta = params[G:G+J]
  gamma = params[G+J:G+2*J]
  theta = params[G+2*J:G+2*J+K]
  sigma = params[-1]
  
  effect_baseline = baseline(trends[D:, :], g)
  effect_short = short_term(regressors[D:, :], beta)
  effect_long = long_term(regressors[:-1, :], gamma)
  effect_exo = exo_term(exo_factors[D:, :], theta)
  
  y_bar = effect_baseline + effect_short + effect_long + effect_exo
  error = y[D:] - tf.squeeze(y_bar)
  
  dist = tfd.Normal(0.0, sigma)
  
  log_likelihood = tf.reduce_sum(tf.math.log(dist.prob(error)))
  l2_penalty = regularization * tf.reduce_sum(tf.abs(params))
  
  total_loss = l2_penalty - log_likelihood
  return total_loss, tf.gradients(total_loss, params)[0]

In [ ]:
# obj_fn = partial(objective_function, **kwargs)

# res = tfp.optimizer.lbfgs_minimize(
#   obj_fn,
#   initial_position=tf.constant([1,2]),
#   tolerance=1e-8
# )